### Log

In [3]:
import datetime, threading
from threading import Lock

module_version = "[1.00]"
log_file_path = "./log.txt"
_log_lock = Lock()

def LOGD(*args, **args2):
    with _log_lock:
        prefix = datetime.datetime.now().strftime('%Y/%m/%d %H:%M:%S.%f') + " " + str(threading.get_ident()) + " " + module_version + '[D]'
        print(prefix, *args, **args2)
        with open(log_file_path, "a") as f:
            print(prefix, *args, **args2, file=f)

def LOGW(*args, **args2):
    with _log_lock:
        prefix = datetime.datetime.now().strftime('%Y/%m/%d %H:%M:%S.%f') + " " + str(threading.get_ident()) + " " + module_version + '[W]'
        print(prefix, *args, **args2)
        with open(log_file_path, "a") as f:
            print(prefix, *args, **args2, file=f)

def LOGE(*args, **args2):
    with _log_lock:
        prefix = datetime.datetime.now().strftime('%Y/%m/%d %H:%M:%S.%f') + " " + str(threading.get_ident()) + " " + module_version + '[E]'
        print(prefix, *args, **args2)
        with open(log_file_path, "a") as f:
            print(prefix, *args, **args2, file=f)

             # YEAR/MM/DD HH:MM:SS.UUUUUU PID   VER  TAG your_msg
LOGD("LOGD") # 2020/05/17 09:01:47.947437 2664 [1.00][D] LOGD
LOGW("LOGW") # 2020/05/17 09:01:47.949437 2664 [1.00][W] LOGW
LOGE("LOGE") # 2020/05/17 09:01:47.950437 2664 [1.00][E] LOGE

2020/05/29 23:24:28.682179 5128 [1.00][D] LOGD
2020/05/29 23:24:28.683179 5128 [1.00][W] LOGW
2020/05/29 23:24:28.684179 5128 [1.00][E] LOGE


### Server + Select

In [8]:
import socket, select, threading, time, traceback

server = 0

def server_thread():
    port = 10030
    
    try:
        global server
        server = socket.socket(socket.AF_INET, socket.SOCK_STREAM) 
        server.setsockopt(socket.SOL_SOCKET, socket.SO_REUSEADDR, 1)
        server.bind(('localhost', port))
        server.listen(5)
        inputs = [server]
    except Exception as err:
        print(traceback.format_exc())
        return
    while True:
        if len(inputs) == 0:
            print("break while loop")
            break
        try:
            readable, _, _ = select.select(inputs, [], [])
            for sck in readable:
                if sck is server:
                    try:
                        client, addr = sck.accept()
                        client.setblocking(0)
                        inputs.append(client)
                        print("remote connected")
                    except Exception as err:
                        inputs.remove(server)
                        print("server is closed")
                else:
                    try:
                        data = sck.recv(1024)
                        sck.send(b'server received: ' + data)
                    except Exception as err: # if remote side close the connection
                        sck.close()
                        inputs.remove(sck)
                        print("remote disconnected")
        except Exception as err:
            print(traceback.format_exc())

threading.Thread(target=server_thread).start()

### Client

In [1]:
import socket

try:
    port = 10030
    client = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
    
    client.connect(('localhost', port))
    
    client.send(b'hello from client')
    data = client.recv(1024)
    print(data)
    
    client.close()
except Exception as err:
    print(traceback.format_exc())

b''


In [10]:
server.close()

server is closed
break while loop


### Multi-Clients

In [9]:
import socket

try:
    port = 10030
    c1 = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
    c2 = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
    
    c1.connect(('localhost', port))
    c2.connect(('localhost', port))
    
    c1.send(b'hello from client 1')
    data = c1.recv(1024)
    print(data)
    
    c2.send(b'hello from client 2')
    data = c2.recv(1024)
    print(data)
    
    c1.close()
    c2.close()
except Exception as err:
    print(traceback.format_exc())

remote connected
remote connected
b'server received: hello from client 1'
b'server received: hello from client 2'
remote disconnected
remote disconnected


### Server + Client in the same select

In [ ]:
import socket, select, threading, time, traceback

port = 10021

# server
server = socket.socket(socket.AF_INET, socket.SOCK_STREAM) 
server.setsockopt(socket.SOL_SOCKET, socket.SO_REUSEADDR, 1)
server.bind(('localhost', port))
server.listen(5)

# client
client = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
client.setblocking(0)

# create other thread for testing
def client_thread():
    time.sleep(0.5)
    ret = client.connect_ex(('localhost', port))
    LOGD(ret)
    time.sleep(0.5)
    client.send(b'hello 1')
    time.sleep(0.5)
    client.send(b'hello 2')
    time.sleep(0.5)
    client.close()    
threading.Thread(target=client_thread).start()

# select
inputs = [server, client]

while True:
    try:
        readable, _, _ = select.select(inputs, [], [])
        for sck in readable:
            if sck is server:
                client, addr = sck.accept()
                client.setblocking(0)
                inputs.append(client)
                LOGD("remote connected, inputs num=", len(inputs))
            elif sck is client:
                try:
                    data = sck.recv(1024)
                    LOGD('client received the data from server:', data)
                except Exception as err: # if remote side close the connection
                    sck.close()
                    inputs.remove(sck)
                    LOGD("client is closed, inputs num=", len(inputs))
            else:
                try:
                    data = sck.recv(1024)
                    sck.send(b'server received: ' + data)
                except Exception as err: # if remote side close the connection
                    sck.close()
                    inputs.remove(sck)
                    LOGD("remote disconnected, inputs num=", len(inputs))
    except Exception as err:
        LOGE(traceback.format_exc())


2020/05/29 23:24:49.085346 1188 [1.00][D] 10035
2020/05/29 23:24:49.095346 5128 [1.00][D] remote connected, inputs num= 3
2020/05/29 23:24:49.595375 5128 [1.00][D] client received the data from server: b'server received: hello 1'
2020/05/29 23:24:50.095403 5128 [1.00][D] client received the data from server: b'server received: hello 2'
2020/05/29 23:24:50.595432 5128 [1.00][D] client is closed, inputs num= 2
2020/05/29 23:24:50.600432 5128 [1.00][D] remote disconnected, inputs num= 1


### Server Class

In [4]:
import socket, traceback, threading, select

class MyServer:
    def __init__(self):
        self.__isbind = False
        self.__inputs = []
        
    def isbind(self):
        return self.__isbind
    
    def bind(self, port):
        """
        return True is success, False is fail
        """
        if self.__isbind:
            return False
        try:
            self.__server = socket.socket(socket.AF_INET, socket.SOCK_STREAM) 
            self.__server.setsockopt(socket.SOL_SOCKET, socket.SO_REUSEADDR, 1)
            self.__server.bind(('localhost', port))
            self.__server.listen(5)
            self.__inputs.append(self.__server)
            self.__thread = threading.Thread(target=self.__select_thread)
            self.__thread.start()
            self.__isbind = True
        except Exception:
            print(traceback.format_exc())
        return self.__isbind
    
    def getconns(self):
        """
        return the connection list
        """
        if not self.__isbind:
            return []
        out = self.__inputs.copy()
        out.remove(self.__server)
        return out
    
    def broadcast(self, data):
        """
        data is a bytes
        return True is success, False is fail
        """
        if not self.__isbind:
            return False
        
        conns = self.getconns()
        try:
            for conn in conns:
                conn.send(data)
        except Exception:
            print(traceback.format_exc())
            return False
        return True

    def close(self):
        """
        return True is success, False is fail
        """
        if not self.__isbind:
            return False
        try:
            for sck in self.__inputs:
                sck.close()
            self.__isbind = False
            self.__thread.join()
            return True
        except Exception:
            print(traceback.format_exc())
            return False
            

    def __select_thread(self):
        exit_while = False
        while not exit_while:
            try:
                readable, _, _ = select.select(self.__inputs, [], [])
                for sck in readable:
                    if sck is self.__server:
                        try:
                            client, addr = sck.accept()
                            client.setblocking(0)
                            self.__inputs.append(client)
                            # remote connected
                        except Exception:
                            self.__inputs.remove(self.__server)
                            # server is closed
                            exit_while = True
                    else:
                        try:
                            data = sck.recv(1024)
                            if len(data) == 0:
                                self.__inputs.remove(sck)
                                # remote disconnected
                            else:
                                print("server recv", data)
                        except (ConnectionAbortedError, ConnectionResetError):
                            sck.close()
                            self.__inputs.remove(sck)
                            # remote disconnected
                        except Exception:
                            print(traceback.format_exc())
                            sck.close()
                            self.__inputs.remove(sck)
                            # remote disconnected
            except ValueError:
                exit_while = True
            except Exception:
                print(traceback.format_exc())
                exit_while = True

### Client Class

In [5]:
import socket, traceback, threading, select, time

class MyClient:
    def __init__(self):
        self.__isconnected = False
        
    def connect(self, ip, port):
        """
        return True is success, False is fail
        """
        if self.__isconnected:
            return False
        try:
            self.__client = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
            self.__client.connect((ip, port))
            self.__thread = threading.Thread(target=self.__client_thread)
            self.__thread.start()
            self.__isconnected = True
        except Exception:
            print(traceback.format_exc())
        return self.__isconnected
    
    def isconnected(self):
        return self.__isconnected
    
    def send(self, data):
        """ 
        return True is success, False is fail
        data is a bytes
        """
        if not self.__isconnected:
            return False
        try:
            self.__client.send(data)
            return True
        except Exception:
            print(traceback.format_exc())
            return False
    
    def close(self):
        """
        return True is success, False is fail
        """
        if not self.__isconnected:
            return False
        try:
            self.__client.close()
            self.__thread.join()
            self.__isconnected = False
            return True
        except Exception:
            print(traceback.format_exc())
            return False

    def __client_thread(self):
        self.__client.setblocking(True)
        while True:
            try:
                data = self.__client.recv(1024)
                if len(data) == 0:
                    break
                print("client recv", data)
            except (ConnectionAbortedError, ConnectionResetError):
                break
            except Exception:
                print(traceback.format_exc())
                break

### Test code

In [8]:
import time

server = MyServer()
server.bind(12345)

client = MyClient()

if client.connect("127.0.0.1", 12345):
    client.send(b'client hello')
    time.sleep(0.2)
    server.broadcast(b'server hello')
    time.sleep(0.5)
    time.sleep(0.5)
    client.close()

#time.sleep(0.5)
server.close()

server recv b'client hello'
client recv b'server hello'


True

### tkinter + socket

In [1]:
#from tkinter import *
#from tkinter.ttk import *
import datetime, socket, traceback, threading, select, time
import tkinter as tk

def keyEvent(event):
    if event.keysym == "Escape":
        global server
        if server.isbind():
            print("server close")
            server.close()
        root.destroy()

# ------------------ log utility ------------------
def get_time_string():
    return datetime.datetime.now().strftime('%H:%M:%S.%f')

def server_log(msg):
    server_log_text["state"] = "normal"
    server_log_text.insert(tk.END, get_time_string() + " " + msg + "\n")
    server_log_text["state"] = "disabled"
    server_log_text.see("end")
    print("[server]", msg)

def client1_log(msg):
    client1_log_text["state"] = "normal"
    client1_log_text.insert(tk.END, get_time_string() + " " + msg + "\n")
    client1_log_text["state"] = "disabled"
    client1_log_text.see("end")
    print("[client 1]", msg)

def client2_log(msg):
    client2_log_text["state"] = "normal"
    client2_log_text.insert(tk.END, get_time_string() + " " + msg + "\n")
    client2_log_text["state"] = "disabled"
    client2_log_text.see("end")
    print("[client 2]", msg)

#port = 30033
# ------------------ Client ------------------
import socket, traceback, threading, select, time

class MyClient:
    def __init__(self):
        self.__isconnected = False
        
    def connect(self, ip, port, logcallback):
        """
        return True is success, False is fail
        """
        if self.__isconnected:
            return False
        try:
            self.__client = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
            self.__client.connect((ip, port))
            self.__thread = threading.Thread(target=self.__client_thread)
            self.__thread.start()
            self.__isconnected = True
            self.__logcallback = logcallback
        except Exception:
            print(traceback.format_exc())
        return self.__isconnected
    
    def isconnected(self):
        return self.__isconnected
    
    def send(self, data):
        """ 
        return True is success, False is fail
        data is a bytes
        """
        if not self.__isconnected:
            return False
        try:
            self.__client.send(data)
            return True
        except Exception:
            print(traceback.format_exc())
            return False
    
    def close(self):
        """
        return True is success, False is fail
        """
        if not self.__isconnected:
            return False
        try:
            self.__client.close()
            # self.__thread.join()
            self.__isconnected = False
            return True
        except Exception:
            print(traceback.format_exc())
            return False

    def __client_thread(self):
        self.__client.setblocking(True)
        while True:
            try:
                data = self.__client.recv(1024)
                if len(data) == 0:
                    self.__logcallback("remote disconnected, read len 0")
                    break
                self.__logcallback("recv %s" % str(data))
            except (ConnectionAbortedError, ConnectionResetError):
                self.__logcallback("remote disconnected, connection broken")
                break
            except Exception:
                print(traceback.format_exc())
                self.__logcallback("remote disconnected, unexpected")
                break

client1 = MyClient()
client2 = MyClient()
# ------------------ Server ------------------
import socket, traceback, threading, select

class MyServer:
    def __init__(self):
        self.__isbind = False
        self.__inputs = []
        
    def isbind(self):
        return self.__isbind
    
    def bind(self, port):
        """
        return True is success, False is fail
        """
        if self.__isbind:
            return False
        try:
            self.__server = socket.socket(socket.AF_INET, socket.SOCK_STREAM) 
            self.__server.setsockopt(socket.SOL_SOCKET, socket.SO_REUSEADDR, 1)
            self.__server.bind(('localhost', port))
            self.__server.listen(5)
            self.__inputs.append(self.__server)
            self.__thread = threading.Thread(target=self.__select_thread)
            self.__thread.start()
            self.__isbind = True
        except Exception:
            print(traceback.format_exc())
        return self.__isbind
    
    def getconns(self):
        """
        return the connection list
        """
        if not self.__isbind:
            return []
        out = self.__inputs.copy()
        out.remove(self.__server)
        return out
    
    def broadcast(self, data):
        """
        data is a bytes
        return True is success, False is fail
        """
        if not self.__isbind:
            return False
        
        conns = self.getconns()
        try:
            for conn in conns:
                conn.send(data)
        except Exception:
            print(traceback.format_exc())
            return False
        return True

    def close(self):
        """
        return True is success, False is fail
        """
        if not self.__isbind:
            return False
        try:
            for sck in self.__inputs:
                sck.close()
            self.__inputs = []
            self.__isbind = False
            #self.__thread.join()
            return True
        except Exception:
            print(traceback.format_exc())
            return False
            

    def __select_thread(self):
        exit_while = False
        while not exit_while:
            try:
                readable, _, _ = select.select(self.__inputs, [], [])
                for sck in readable:
                    if sck is self.__server:
                        try:
                            client, addr = sck.accept()
                            client.setblocking(0)
                            self.__inputs.append(client)
                            server_log("remote connected")
                        except Exception:
                            self.__inputs.remove(self.__server)
                            # server is closed
                            exit_while = True
                    else:
                        try:
                            data = sck.recv(1024)
                            if len(data) == 0:
                                self.__inputs.remove(sck)
                                server_log("remote disconnected, read len 0")
                            else:
                                print("server recv", data)
                                server_log("recv %s" % str(data))
                        except (ConnectionAbortedError, ConnectionResetError):
                            sck.close()
                            self.__inputs.remove(sck)
                            server_log("remote disconnected, connection broken")
                        except Exception:
                            print(traceback.format_exc())
                            sck.close()
                            self.__inputs.remove(sck)
                            server_log("remote disconnected, unexpcted error")
            except ValueError:
                exit_while = True
            except Exception:
                print(traceback.format_exc())
                exit_while = True
        server_log("exit while loop")
                
server = MyServer()
# ------------------ UI Event Handle ------------------
def server_bind_cb_click():
    global server
    if server_bind_cb_var.get() == 1:
        try:
            port = int(server_entry_port.get())
            if not server.bind(port):
                assert False, "bind failed"
            else:
                server_log("bind %d success" % port)
        except Exception as err:
            server_log("ERR: reason=[%s]" % (str(err)))
            server_bind_cb_var.set(0)
    else:
        try:
            if not server.close():
                assert False,"close failed"
            else:
                server_log("unbind success")
        except Exception as err:
            server_log("ERR: server unbind failed , reason=[%s]" % str(err))
            server_bind_cb_var.set(1)

def server_write_click():
    global server
    if server.isbind():
        server.broadcast(server_entry_write.get().encode())
    else:
        server_log("please bind server before write")

def client1_connect_cb_click():
    global client1
    if client1_connect_cb_var.get() == 1:
        try:
            ip = client1_entry_fqdn.get()
            port = int(client1_entry_port.get())
            client1_log("client connection addr=[%s] port=[%d]" % (ip, port))
            if not client1.connect(ip, port, client1_log):
                assert False, "connection failed"
            else:
                client1_log("connection success")
        except Exception as err:
            client1_log("ERR: reason=[%s]" % (str(err)))
            client1_connect_cb_var.set(0)
    else:
        if not client1.close():
            client1_connect_cb_var("ERR: client disconnection failed")
            client1_connect_cb_var.set(1)

def client1_write_click():
    global client1
    if not client1.send(b'client1 hello'):
        client1_log("ERR: write failed")

def client2_connect_cb_click():
    global client2
    if client2_connect_cb_var.get() == 1:
        try:
            ip = client2_entry_fqdn.get()
            port = int(client2_entry_port.get())
            client2_log("client connection addr=[%s] port=[%d]" % (ip, port))
            if not client2.connect(ip, port, client2_log):
                assert False, "connection failed"
            else:
                client2_log("connection success")
            pass
        except Exception as err:
            client2_log("ERR: reason=[%s]" % (str(err)))
            client2_connect_cb_var.set(0)
    else:
        if not client2.close():
            client2_connect_cb_var("ERR: client disconnection failed")
            client2_connect_cb_var.set(1)

def client2_write_click():
    global client2
    if not client2.send(b'client2 hello'):
        client2_log("ERR: write failed")

# ------------------ Main UI ------------------
root = tk.Tk()

panel = tk.PanedWindow(orient=tk.HORIZONTAL)
panel.pack()
# ------------------ UI - Server ------------------
server_panel = tk.PanedWindow(orient=tk.VERTICAL)
server_panel.add(tk.Label(root, text="Server", anchor=tk.CENTER, font=("Helvetic", 12, "bold")))

server_panel_port = tk.PanedWindow(orient=tk.HORIZONTAL)
server_panel_port.add(tk.Label(root, text="Port", width=5))
server_entry_port = tk.Entry(root, width=50)
server_entry_port.insert(0, "22222")
server_panel_port.add(server_entry_port)
server_bind_cb_var = tk.IntVar()
server_panel_port.add(tk.Checkbutton(root, text="bind", variable=server_bind_cb_var, command=server_bind_cb_click))
server_panel.add(server_panel_port)

server_panel_write = tk.PanedWindow(orient=tk.HORIZONTAL)
server_panel_write.add(tk.Label(root, text="Write", width=5))
server_entry_write = tk.Entry(root, width=50)
server_entry_write.insert(0, "server write")
server_panel_write.add(server_entry_write)
server_panel_write.add(tk.Button(root, text="send", command=server_write_click))
server_panel.add(server_panel_write)

server_log_text = tk.Text(root, width=50, height=50, wrap="none", borderwidth=0, undo=True)
server_log_text.insert(tk.END, "server's log\n")
server_log_text["state"] = "disabled"
server_panel.add(server_log_text)
# ------------------ UI - Client 1 ------------------
client1_panel = tk.PanedWindow(orient=tk.VERTICAL)
client1_panel.add(tk.Label(root, text="Client 1", anchor=tk.CENTER, font=("Helvetic", 12, "bold")))

client1_panel_addr = tk.PanedWindow(orient=tk.HORIZONTAL)
client1_panel_addr.add(tk.Label(root, text="FQDN or IP", width=10))
client1_entry_fqdn = tk.Entry(root, width=50)
client1_entry_fqdn.insert(0, "localhost")
client1_panel_addr.add(client1_entry_fqdn)
client1_panel.add(client1_panel_addr)

client1_panel_port = tk.PanedWindow(orient=tk.HORIZONTAL)
client1_panel_port.add(tk.Label(root, text="Port", width=10))
client1_entry_port = tk.Entry(root, width=50)
client1_entry_port.insert(0, "22222")
client1_panel_port.add(client1_entry_port)
client1_connect_cb_var = tk.IntVar()
client1_panel_port.add(tk.Checkbutton(root, text="connect", variable=client1_connect_cb_var, command=client1_connect_cb_click))
client1_panel.add(client1_panel_port)

client1_panel_write = tk.PanedWindow(orient=tk.HORIZONTAL)
client1_panel_write.add(tk.Label(root, text="Write", width=10))
client1_entry_write = tk.Entry(root, width=50)
client1_entry_write.insert(0, "client 1write")
client1_panel_write.add(client1_entry_write)
client1_panel_write.add(tk.Button(root, text="send", command=client1_write_click))
client1_panel.add(client1_panel_write)

client1_log_text = tk.Text(root, width=50, height=50, wrap="none", borderwidth=0, undo=True)
client1_log_text.insert(tk.END, "client1's log\n")
client1_log_text["state"] = "disabled"
client1_panel.add(client1_log_text)
# ------------------ UI - Client 2 ------------------
client2_panel = tk.PanedWindow(orient=tk.VERTICAL)
client2_panel.add(tk.Label(root, text="Client 2", anchor=tk.CENTER, font=("Helvetic", 12, "bold")))

client2_panel_addr = tk.PanedWindow(orient=tk.HORIZONTAL)
client2_panel_addr.add(tk.Label(root, text="FQDN or IP", width=10))
client2_entry_fqdn = tk.Entry(root, width=50)
client2_entry_fqdn.insert(0, "localhost")
client2_panel_addr.add(client2_entry_fqdn)
client2_panel.add(client2_panel_addr)

client2_panel_port = tk.PanedWindow(orient=tk.HORIZONTAL)
client2_panel_port.add(tk.Label(root, text="Port", width=10))
client2_entry_port = tk.Entry(root, width=50)
client2_entry_port.insert(0, "22222")
client2_panel_port.add(client2_entry_port)
client2_connect_cb_var = tk.IntVar()
client2_panel_port.add(tk.Checkbutton(root, text="connect", variable=client2_connect_cb_var, command=client2_connect_cb_click))
client2_panel.add(client2_panel_port)

client2_panel_write = tk.PanedWindow(orient=tk.HORIZONTAL)
client2_panel_write.add(tk.Label(root, text="Write", width=10))
client2_entry_write = tk.Entry(root, width=50)
client2_entry_write.insert(0, "client2 write")
client2_panel_write.add(client2_entry_write)
client2_panel_write.add(tk.Button(root, text="send", command=client2_write_click))
client2_panel.add(client2_panel_write)

client2_log_text = tk.Text(root, width=50, height=50, wrap="none", borderwidth=0, undo=True)
client2_log_text.insert(tk.END, "client2's log\n")
client2_log_text["state"] = "disabled"
client2_panel.add(client2_log_text)
# ------------------ UI - Horizontal Layout ------------------
panel.add(server_panel)
panel.add(tk.PanedWindow(root, orient=tk.VERTICAL, bg="red"))
panel.add(client1_panel)
panel.add(tk.PanedWindow(root, orient=tk.VERTICAL, bg="red"))
panel.add(client2_panel)
# -------------------------------------
root.bind_all("<KeyPress>", keyEvent)
root.geometry('1600x500+0+0')
root.lift()
root.focus_force()
root.mainloop()

[server] bind 22222 success
[server] unbind success
[server] exit while loop
[server] bind 22222 success
[client 1] client connection addr=[localhost] port=[22222]
[client 1] connection success
[server] remote connected
[client 2] client connection addr=[localhost] port=[22222]
[client 2] connection success
[server] remote connected
[server] unbind success
[server] exit while loop
[client 2][client 1]  remote disconnected, read len 0
remote disconnected, read len 0
[server] bind 22222 success
[client 2] client connection addr=[localhost] port=[22222]
[client 2] connection success
[server] remote connected
[client 2] [server]remote disconnected, connection broken
 remote disconnected, connection broken
[client 2] client connection addr=[localhost] port=[22222]
[client 2] connection success
[server] remote connected
[client 2] remote disconnected, connection broken
[server] remote disconnected, connection broken
[client 2] client connection addr=[localhost] port=[22222]
[client 2] connec